# Numerical Integration - Single Variable
- **Purpose**: Compute definite integrals when analytical solutions are difficult/impossible
- **scipy.integrate**: Adaptive quadrature, Gaussian quadrature, Romberg integration
- **Applications**: Area under curves, probability, physics problems, statistics

Key methods:
- **quad()**: General-purpose adaptive integration (most common)
- **quadrature()**: Gaussian quadrature
- **romberg()**: Romberg integration (extrapolation)
- **fixed_quad()**: Fixed-order Gaussian quadrature
- Specialized: trapezoid, Simpson's rule

In [1]:
import numpy as np
from scipy import integrate
import matplotlib.pyplot as plt

# Set print options
np.set_printoptions(precision=6, suppress=True)

print("Numerical integration module loaded")

Numerical integration module loaded


## Definite Integral

Compute:

\[ I = \int_a^b f(x) \, dx \]

**Interpretation**:
- Area under curve f(x) from x=a to x=b
- Accumulation of f(x) over interval [a, b]

**Challenges**:
- No closed-form antiderivative
- Complex functions
- Singularities or discontinuities

**Example**: \( \int_0^1 e^{-x^2} dx \) has no elementary antiderivative

## quad() - General Purpose Integration

**Function**: `scipy.integrate.quad(func, a, b, args=())`

**Returns**: `(result, error_estimate)`

**Parameters**:
- `func`: Function to integrate (must return scalar)
- `a, b`: Integration limits (can be ±∞)
- `args`: Extra arguments to pass to func
- `epsabs`, `epsrel`: Absolute and relative error tolerances

**Algorithm**: Adaptive Gauss-Kronrod quadrature (QUADPACK)

In [2]:
# Example 1: ∫₀¹ x² dx = [x³/3]₀¹ = 1/3
result, error = integrate.quad(lambda x: x**2, 0, 1)
print("Example 1: ∫₀¹ x² dx")
print(f"  Result: {result:.10f}")
print(f"  Expected: {1/3:.10f}")
print(f"  Error estimate: {error:.2e}")

# Example 2: ∫₀^π sin(x) dx = [-cos(x)]₀^π = 2
result, error = integrate.quad(np.sin, 0, np.pi)
print("\nExample 2: ∫₀^π sin(x) dx")
print(f"  Result: {result:.10f}")
print(f"  Expected: 2.0")
print(f"  Error estimate: {error:.2e}")

# Example 3: ∫₀¹ e^x dx = [e^x]₀¹ = e - 1
result, error = integrate.quad(np.exp, 0, 1)
print("\nExample 3: ∫₀¹ e^x dx")
print(f"  Result: {result:.10f}")
print(f"  Expected: {np.e - 1:.10f}")
print(f"  Error estimate: {error:.2e}")

Example 1: ∫₀¹ x² dx
  Result: 0.3333333333
  Expected: 0.3333333333
  Error estimate: 3.70e-15

Example 2: ∫₀^π sin(x) dx
  Result: 2.0000000000
  Expected: 2.0
  Error estimate: 2.22e-14

Example 3: ∫₀¹ e^x dx
  Result: 1.7182818285
  Expected: 1.7182818285
  Error estimate: 1.91e-14


## Real Example: Gaussian (Normal) Distribution

The Gaussian function has **no elementary antiderivative**:

\[ f(x) = e^{-x^2} \]

Related to the error function and probability:

\[ \int_{-\infty}^{\infty} e^{-x^2} dx = \sqrt{\pi} \]

This integral appears in:
- **Statistics**: Normal distribution
- **Quantum mechanics**: Wavefunction normalization
- **Optics**: Gaussian beams

In [3]:
# Gaussian function
def gaussian(x):
    return np.exp(-x**2)

# Integrate from -∞ to ∞
result, error = integrate.quad(gaussian, -np.inf, np.inf)

print("Gaussian integral: ∫₋∞^∞ e^(-x²) dx")
print(f"  Numerical result: {result:.10f}")
print(f"  Expected (√π): {np.sqrt(np.pi):.10f}")
print(f"  Error estimate: {error:.2e}")

# Standard normal distribution: ∫₀^z (1/√(2π)) e^(-x²/2) dx
def standard_normal(x):
    return (1/np.sqrt(2*np.pi)) * np.exp(-x**2/2)

# Probability that Z < 1 (about 84% for standard normal)
prob, error = integrate.quad(standard_normal, -np.inf, 1)
print("\nStandard Normal P(Z < 1):")
print(f"  Probability: {prob:.6f}")
print(f"  Expected: ~0.841345")

# Verify total probability = 1
total_prob, _ = integrate.quad(standard_normal, -np.inf, np.inf)
print(f"\nTotal probability: {total_prob:.10f} (should be 1.0)")

Gaussian integral: ∫₋∞^∞ e^(-x²) dx
  Numerical result: 1.7724538509
  Expected (√π): 1.7724538509
  Error estimate: 1.42e-08

Standard Normal P(Z < 1):
  Probability: 0.841345
  Expected: ~0.841345

Total probability: 1.0000000000 (should be 1.0)


## Physics Example: Work Done by Variable Force

Work done by force F(x) moving from x=a to x=b:

\[ W = \int_a^b F(x) \, dx \]

**Example**: Spring force F(x) = -kx (Hooke's law)

Work to compress spring from 0 to distance d:

\[ W = \int_0^d kx \, dx = \frac{1}{2}kd^2 \]

Let's compute for non-linear spring: F(x) = -kx - αx³

In [4]:
# Non-linear spring force F(x) = -(kx + αx³)
k = 100  # N/m (spring constant)
alpha = 50  # N/m³ (non-linear term)
d = 0.5  # compress by 0.5 meters

def spring_force(x, k, alpha):
    """Magnitude of restoring force"""
    return k*x + alpha*x**3

# Work = ∫₀^d F(x) dx
work, error = integrate.quad(spring_force, 0, d, args=(k, alpha))

print(f"Non-linear spring compression")
print(f"  Spring constant k: {k} N/m")
print(f"  Non-linear term α: {alpha} N/m³")
print(f"  Compression distance: {d} m")
print(f"\nWork required: {work:.4f} Joules")

# Compare with linear spring (α = 0)
work_linear = 0.5 * k * d**2
print(f"\nLinear spring (α=0): {work_linear:.4f} Joules")
print(f"Extra work due to non-linearity: {work - work_linear:.4f} Joules")

# Analytical solution: kd²/2 + αd⁴/4
work_analytical = k*d**2/2 + alpha*d**4/4
print(f"\nAnalytical result: {work_analytical:.4f} Joules")
print(f"Numerical error: {abs(work - work_analytical):.2e} Joules")

Non-linear spring compression
  Spring constant k: 100 N/m
  Non-linear term α: 50 N/m³
  Compression distance: 0.5 m

Work required: 13.2812 Joules

Linear spring (α=0): 12.5000 Joules
Extra work due to non-linearity: 0.7812 Joules

Analytical result: 13.2812 Joules
Numerical error: 0.00e+00 Joules


## Oscillating Functions

Functions with rapid oscillations can be challenging:

**Example**: Fresnel integral (optics)

\[ S(x) = \int_0^x \sin(t^2) \, dt \]

Used in:
- **Wave optics**: Diffraction patterns
- **Signal processing**: Chirp signals
- **Road design**: Transition curves

In [5]:
# Fresnel sine integral
def fresnel_integrand(t):
    return np.sin(t**2)

# Compute for several values
x_values = [1, 2, 3, 5]

print("Fresnel sine integral S(x) = ∫₀^x sin(t²) dt\n")
for x in x_values:
    result, error = integrate.quad(fresnel_integrand, 0, x)
    print(f"  S({x}) = {result:.6f} (error: {error:.2e})")

# Highly oscillating integral: ∫₀^10 sin(20x) dx
result, error = integrate.quad(lambda x: np.sin(20*x), 0, 10)
print(f"\n\nHighly oscillating: ∫₀^10 sin(20x) dx")
print(f"  Result: {result:.10f}")
print(f"  Expected: {(1-np.cos(200))/20:.10f}")
print(f"  Error: {error:.2e}")
print("\nNote: quad() handles oscillating functions well!")

Fresnel sine integral S(x) = ∫₀^x sin(t²) dt

  S(1) = 0.310268 (error: 3.44e-15)
  S(2) = 0.804776 (error: 2.87e-13)
  S(3) = 0.773563 (error: 1.19e-13)
  S(5) = 0.527917 (error: 1.89e-09)


Highly oscillating: ∫₀^10 sin(20x) dx
  Result: 0.0256406162
  Expected: 0.0256406162
  Error: 2.38e-09

Note: quad() handles oscillating functions well!


## Integrating Functions with Singularities

Some integrals have singularities but are still **integrable**:

**Example 1**: \( \int_0^1 \frac{1}{\sqrt{x}} dx = 2 \)

Singularity at x=0, but area is finite

**Example 2**: \( \int_0^1 \ln(x) dx = -1 \)

ln(0) = -∞, but integral converges

**Parameter**: Use `points` to specify known singularities

In [6]:
# Example 1: ∫₀¹ 1/√x dx = [2√x]₀¹ = 2
result, error = integrate.quad(lambda x: 1/np.sqrt(x), 0, 1)
print("Example 1: ∫₀¹ 1/√x dx")
print(f"  Result: {result:.10f}")
print(f"  Expected: 2.0")
print(f"  Error: {error:.2e}")

# Example 2: ∫₀¹ ln(x) dx = [x·ln(x) - x]₀¹ = -1
result, error = integrate.quad(lambda x: np.log(x), 0, 1)
print("\nExample 2: ∫₀¹ ln(x) dx")
print(f"  Result: {result:.10f}")
print(f"  Expected: -1.0")
print(f"  Error: {error:.2e}")

# Example 3: ∫₀¹ x^(-1/4) dx = [4x^(3/4)/3]₀¹ = 4/3
result, error = integrate.quad(lambda x: x**(-0.25), 0, 1)
print("\nExample 3: ∫₀¹ x^(-1/4) dx")
print(f"  Result: {result:.10f}")
print(f"  Expected: {4/3:.10f}")
print(f"  Error: {error:.2e}")
print("\nNote: quad() automatically handles endpoint singularities!")

Example 1: ∫₀¹ 1/√x dx
  Result: 2.0000000000
  Expected: 2.0
  Error: 3.77e-15

Example 2: ∫₀¹ ln(x) dx
  Result: -1.0000000000
  Expected: -1.0
  Error: 1.11e-15

Example 3: ∫₀¹ x^(-1/4) dx
  Result: 1.3333333333
  Expected: 1.3333333333
  Error: 2.22e-15

Note: quad() automatically handles endpoint singularities!


## Integration with Infinite Limits

Use `np.inf` or `-np.inf` for infinite limits:

**Example 1**: Exponential decay
\[ \int_0^{\infty} e^{-x} dx = 1 \]

**Example 2**: Gamma function
\[ \Gamma(n) = \int_0^{\infty} t^{n-1}e^{-t} dt \]

For n=2: Γ(2) = 1! = 1

**Example 3**: Laplace distribution
\[ \int_{-\infty}^{\infty} \frac{1}{2}e^{-|x|} dx = 1 \]

In [7]:
# Example 1: ∫₀^∞ e^(-x) dx = 1
result, error = integrate.quad(lambda x: np.exp(-x), 0, np.inf)
print("Example 1: ∫₀^∞ e^(-x) dx")
print(f"  Result: {result:.10f}")
print(f"  Expected: 1.0")
print(f"  Error: {error:.2e}")

# Example 2: Gamma function Γ(3) = 2! = 2
def gamma_integrand(t, n):
    return t**(n-1) * np.exp(-t)

n = 3
result, error = integrate.quad(gamma_integrand, 0, np.inf, args=(n,))
print(f"\nExample 2: Γ({n}) = ∫₀^∞ t^{n-1}·e^(-t) dt")
print(f"  Result: {result:.10f}")
print(f"  Expected: {np.math.factorial(n-1):.10f}")
print(f"  Error: {error:.2e}")

# Example 3: Laplace distribution normalization
result, error = integrate.quad(lambda x: 0.5*np.exp(-np.abs(x)), -np.inf, np.inf)
print("\nExample 3: ∫₋∞^∞ (1/2)e^(-|x|) dx")
print(f"  Result: {result:.10f}")
print(f"  Expected: 1.0 (normalized distribution)")
print(f"  Error: {error:.2e}")

# Example 4: Standard normal variance: ∫₋∞^∞ x²·φ(x) dx = 1
def normal_variance_integrand(x):
    return x**2 * (1/np.sqrt(2*np.pi)) * np.exp(-x**2/2)

result, error = integrate.quad(normal_variance_integrand, -np.inf, np.inf)
print("\nExample 4: Variance of standard normal")
print(f"  Result: {result:.10f}")
print(f"  Expected: 1.0")
print(f"  Error: {error:.2e}")

Example 1: ∫₀^∞ e^(-x) dx
  Result: 1.0000000000
  Expected: 1.0
  Error: 5.84e-11

Example 2: Γ(3) = ∫₀^∞ t^2·e^(-t) dt
  Result: 2.0000000000


AttributeError: module 'numpy' has no attribute 'math'

## Functions with Parameters

Pass additional arguments using `args` parameter:

```python
def f(x, a, b, c):
    return a*x**2 + b*x + c

result, error = quad(f, 0, 1, args=(2, 3, 4))
```

**Application**: Parametric probability distributions

In [8]:
# Exponential distribution: f(x; λ) = λe^(-λx) for x ≥ 0
def exponential_pdf(x, lam):
    return lam * np.exp(-lam * x)

# Mean of exponential distribution = 1/λ
def exponential_mean_integrand(x, lam):
    return x * lam * np.exp(-lam * x)

lam = 2.0

# Verify normalization
norm, _ = integrate.quad(exponential_pdf, 0, np.inf, args=(lam,))
print(f"Exponential distribution (λ={lam})")
print(f"  Normalization: {norm:.10f} (should be 1.0)")

# Compute mean
mean, _ = integrate.quad(exponential_mean_integrand, 0, np.inf, args=(lam,))
print(f"  Mean: {mean:.10f}")
print(f"  Expected (1/λ): {1/lam:.10f}")

# Compute P(X < 2) = ∫₀² λe^(-λx) dx
prob, _ = integrate.quad(exponential_pdf, 0, 2, args=(lam,))
print(f"  P(X < 2): {prob:.6f}")
print(f"  Expected (1-e^(-4)): {1-np.exp(-4):.6f}")

# Beta distribution example: Beta(2, 3)
def beta_pdf(x, alpha, beta):
    from scipy.special import gamma
    return (gamma(alpha+beta)/(gamma(alpha)*gamma(beta))) * x**(alpha-1) * (1-x)**(beta-1)

alpha, beta_param = 2, 3
norm, _ = integrate.quad(beta_pdf, 0, 1, args=(alpha, beta_param))
print(f"\nBeta({alpha}, {beta_param}) normalization: {norm:.10f}")

Exponential distribution (λ=2.0)
  Normalization: 1.0000000000 (should be 1.0)
  Mean: 0.5000000000
  Expected (1/λ): 0.5000000000
  P(X < 2): 0.981684
  Expected (1-e^(-4)): 0.981684

Beta(2, 3) normalization: 1.0000000000


## Controlling Integration Error

Control accuracy with tolerance parameters:

- **epsabs**: Absolute error tolerance (default: 1.49e-8)
- **epsrel**: Relative error tolerance (default: 1.49e-8)

Integration stops when: `|error| < max(epsabs, epsrel * |result|)`

```python
result, error = quad(f, a, b, epsabs=1e-12, epsrel=1e-12)
```

**Trade-off**: Higher accuracy = more function evaluations

In [9]:
# Function to integrate
def f(x):
    return np.sin(x) / x if x != 0 else 1.0

# Default tolerance
result1, error1 = integrate.quad(f, 0, 10)
print("Default tolerance (≈1.5e-8):")
print(f"  Result: {result1:.15f}")
print(f"  Error estimate: {error1:.2e}")

# Higher accuracy
result2, error2 = integrate.quad(f, 0, 10, epsabs=1e-12, epsrel=1e-12)
print("\nHigh accuracy (1e-12):")
print(f"  Result: {result2:.15f}")
print(f"  Error estimate: {error2:.2e}")

# Lower accuracy (faster)
result3, error3 = integrate.quad(f, 0, 10, epsabs=1e-4, epsrel=1e-4)
print("\nLower accuracy (1e-4):")
print(f"  Result: {result3:.15f}")
print(f"  Error estimate: {error3:.2e}")

print("\nDifference high vs default: {:.2e}".format(abs(result2 - result1)))
print("Difference default vs low: {:.2e}".format(abs(result1 - result3)))

Default tolerance (≈1.5e-8):
  Result: 1.658347594218874
  Error estimate: 2.86e-14

High accuracy (1e-12):
  Result: 1.658347594218874
  Error estimate: 2.86e-14

Lower accuracy (1e-4):
  Result: 1.658347594218874
  Error estimate: 2.86e-14

Difference high vs default: 0.00e+00
Difference default vs low: 0.00e+00


## Gaussian Quadrature

Uses optimally chosen points and weights:

\[ \int_a^b f(x) dx \approx \sum_{i=1}^n w_i f(x_i) \]

**Function**: `scipy.integrate.fixed_quad(func, a, b, n=5)`

- Fixed number of evaluation points (n)
- Exact for polynomials up to degree 2n-1
- Fast for smooth functions

**Comparison**:
- `quad()`: Adaptive (variable # points)
- `fixed_quad()`: Fixed n points
- `quadrature()`: Increases n until convergence

In [10]:
# Function: ∫₀¹ x⁴ dx = 1/5 = 0.2
f = lambda x: x**4

print("Integration of f(x) = x⁴ from 0 to 1\n")

# Fixed quadrature with different orders
for n in [2, 3, 5, 10]:
    result, _ = integrate.fixed_quad(f, 0, 1, n=n)
    error = abs(result - 0.2)
    print(f"  n={n:2d} points: {result:.15f}  (error: {error:.2e})")

# Adaptive quadrature
result_adaptive, error = integrate.quadrature(f, 0, 1)
print(f"\nAdaptive (quadrature): {result_adaptive:.15f}  (error: {error:.2e})")

# quad() for comparison
result_quad, error = integrate.quad(f, 0, 1)
print(f"Adaptive (quad):       {result_quad:.15f}  (error: {error:.2e})")

print(f"\nExact answer: 0.200000000000000")
print("\nNote: Gaussian quadrature is exact for polynomials!")
print("n=3 is exact because x⁴ has degree 4, and 2*3-1 = 5 > 4")

Integration of f(x) = x⁴ from 0 to 1

  n= 2 points: 0.194444444444444  (error: 5.56e-03)
  n= 3 points: 0.200000000000000  (error: 5.55e-17)
  n= 5 points: 0.200000000000000  (error: 8.33e-17)
  n=10 points: 0.200000000000000  (error: 5.55e-17)


AttributeError: module 'scipy.integrate' has no attribute 'quadrature'

## Romberg Integration

Richardson extrapolation applied to trapezoidal rule:

**Function**: `scipy.integrate.romberg(func, a, b, show=False)`

**Algorithm**:
1. Compute trapezoidal rule with increasing refinement
2. Apply Richardson extrapolation to eliminate errors
3. Continue until convergence

**Best for**: Smooth functions on finite intervals

**Advantage**: Shows convergence table (if show=True)

In [11]:
# Function: ∫₀^π sin(x) dx = 2
result = integrate.romberg(np.sin, 0, np.pi, show=True)
print(f"\nFinal result: {result:.15f}")
print(f"Expected: 2.0")
print(f"Error: {abs(result - 2.0):.2e}")

# Another example: ∫₀¹ √x dx = 2/3
print("\n" + "="*60)
result2 = integrate.romberg(np.sqrt, 0, 1, show=True)
print(f"\nFinal result: {result2:.15f}")
print(f"Expected: {2/3:.15f}")
print(f"Error: {abs(result2 - 2/3):.2e}")

AttributeError: module 'scipy.integrate' has no attribute 'romberg'

## Method Comparison

| Method | Best For | Speed | Accuracy Control |
|--------|----------|-------|------------------|
| `quad()` | General purpose, any function | Medium | epsabs, epsrel |
| `fixed_quad()` | Smooth functions, fixed cost | Fast | n (order) |
| `quadrature()` | Smooth functions, adaptive | Medium | tol, maxiter |
| `romberg()` | Very smooth, finite intervals | Slow | tol, rtol |
| `simpson()` | Uniformly spaced data | Fast | dx (spacing) |
| `trapezoid()` | Uniformly spaced data | Fastest | dx (spacing) |

**Recommendation**: Use `quad()` unless you have specific needs

In [12]:
import time

# Test function: ∫₀² e^(-x²) dx
f = lambda x: np.exp(-x**2)
a, b = 0, 2

print("Integration of e^(-x²) from 0 to 2\n")
print(f"{'Method':<20} {'Result':<18} {'Time (ms)':<12}")
print("="*50)

# quad()
t0 = time.time()
result1, _ = integrate.quad(f, a, b)
t1 = (time.time() - t0) * 1000
print(f"{'quad()':<20} {result1:.15f} {t1:.3f}")

# fixed_quad() n=10
t0 = time.time()
result2, _ = integrate.fixed_quad(f, a, b, n=10)
t2 = (time.time() - t0) * 1000
print(f"{'fixed_quad(n=10)':<20} {result2:.15f} {t2:.3f}")

# quadrature()
t0 = time.time()
result3, _ = integrate.quadrature(f, a, b)
t3 = (time.time() - t0) * 1000
print(f"{'quadrature()':<20} {result3:.15f} {t3:.3f}")

# romberg()
t0 = time.time()
result4 = integrate.romberg(f, a, b, show=False)
t4 = (time.time() - t0) * 1000
print(f"{'romberg()':<20} {result4:.15f} {t4:.3f}")

print("\nAll methods agree to within numerical precision!")
print(f"Max difference: {max([abs(result1-r) for r in [result2, result3, result4]]):.2e}")

Integration of e^(-x²) from 0 to 2

Method               Result             Time (ms)   
quad()               0.882081390762422 0.106
fixed_quad(n=10)     0.882081390762108 0.108


AttributeError: module 'scipy.integrate' has no attribute 'quadrature'

## Practical Example: Arc Length of Curve

Length of curve y = f(x) from x=a to x=b:

\[ L = \int_a^b \sqrt{1 + \left(\frac{dy}{dx}\right)^2} dx \]

**Example**: Parabola y = x² from x=0 to x=2

- f(x) = x²
- f'(x) = 2x
- Arc length = \( \int_0^2 \sqrt{1 + 4x^2} dx \)

In [13]:
# Arc length of y = x² from 0 to 2
def arc_length_integrand(x):
    # dy/dx = 2x
    dydx = 2*x
    return np.sqrt(1 + dydx**2)

length, error = integrate.quad(arc_length_integrand, 0, 2)

print("Arc length of y = x² from x=0 to x=2")
print(f"  Numerical result: {length:.10f}")
print(f"  Error estimate: {error:.2e}")

# Compare with straight-line distance
straight_distance = np.sqrt((2-0)**2 + (4-0)**2)
print(f"\n  Straight-line distance: {straight_distance:.10f}")
print(f"  Extra distance due to curve: {length - straight_distance:.10f}")
print(f"  Percentage increase: {100*(length/straight_distance - 1):.2f}%")

# Another example: semicircle y = √(1-x²) from -1 to 1
def semicircle_arc_integrand(x):
    # dy/dx = -x/√(1-x²)
    if abs(x) >= 1:
        return 1e10  # Should not reach here
    dydx = -x / np.sqrt(1 - x**2)
    return np.sqrt(1 + dydx**2)

# Simplifies to: √(1 + x²/(1-x²)) = 1/√(1-x²)
def semicircle_arc_simple(x):
    return 1.0 / np.sqrt(1 - x**2)

length_circle, _ = integrate.quad(semicircle_arc_simple, -1, 1)
print(f"\n\nArc length of semicircle y = √(1-x²):")
print(f"  Numerical result: {length_circle:.10f}")
print(f"  Expected (πr = π): {np.pi:.10f}")
print(f"  Error: {abs(length_circle - np.pi):.2e}")

Arc length of y = x² from x=0 to x=2
  Numerical result: 4.6467837624
  Error estimate: 1.57e-09

  Straight-line distance: 4.4721359550
  Extra distance due to curve: 0.1746478074
  Percentage increase: 3.91%


Arc length of semicircle y = √(1-x²):
  Numerical result: 3.1415926536
  Expected (πr = π): 3.1415926536
  Error: 2.03e-13


## Summary: Key Functions

### Main Integration Functions:

| Function | Syntax | Best For | Returns |
|----------|--------|----------|----------|
| `quad()` | `quad(f, a, b)` | General purpose | (result, error) |
| `fixed_quad()` | `fixed_quad(f, a, b, n)` | Fixed order | (result, None) |
| `quadrature()` | `quadrature(f, a, b)` | Adaptive Gaussian | (result, error) |
| `romberg()` | `romberg(f, a, b)` | Very smooth | result |

### Special Features:

✓ **Infinite limits**: Use `np.inf` or `-np.inf`  
✓ **Singularities**: quad() handles endpoint singularities automatically  
✓ **Parameters**: Pass extra args with `args=(param1, param2)`  
✓ **Error control**: Set `epsabs` and `epsrel` for accuracy  
✓ **Oscillating**: quad() uses adaptive mesh refinement  

### Common Applications:
- **Probability**: ∫ PDF(x) dx for cumulative distribution
- **Physics**: Work = ∫ F(x) dx, arc length, energy
- **Statistics**: Moments = ∫ x^n · f(x) dx
- **Engineering**: Area, volume, center of mass

## Quick Reference Examples

```python
# Basic integral
result, error = integrate.quad(lambda x: x**2, 0, 1)

# With parameters
def f(x, a, b):
    return a*x + b
result, error = integrate.quad(f, 0, 1, args=(2, 3))

# Infinite limits
result, error = integrate.quad(lambda x: np.exp(-x), 0, np.inf)

# High accuracy
result, error = integrate.quad(f, 0, 1, epsabs=1e-12, epsrel=1e-12)

# Fixed order Gaussian
result, _ = integrate.fixed_quad(f, 0, 1, n=10)

# Romberg with convergence table
result = integrate.romberg(f, 0, 1, show=True)
```

## Practice Problems

Try computing these integrals:

1. **Normal CDF**: \( P(Z < 2) = \int_{-\infty}^2 \frac{1}{\sqrt{2\pi}}e^{-x^2/2} dx \)

2. **Sine integral**: \( \text{Si}(\pi) = \int_0^\pi \frac{\sin(x)}{x} dx \)

3. **Elliptic integral**: \( K(0.5) = \int_0^{\pi/2} \frac{1}{\sqrt{1 - 0.5\sin^2(\theta)}} d\theta \)

4. **Arc length**: Length of y = sin(x) from 0 to π

5. **Expected value**: E[X] for exponential with λ=3

Solutions in next cell!

In [14]:
print("PRACTICE PROBLEM SOLUTIONS\n")
print("="*60)

# 1. Normal CDF P(Z < 2)
normal_pdf = lambda x: (1/np.sqrt(2*np.pi)) * np.exp(-x**2/2)
prob, _ = integrate.quad(normal_pdf, -np.inf, 2)
print(f"\n1. P(Z < 2) = {prob:.6f}")
print(f"   (About 97.7% of data within 2σ)")

# 2. Sine integral Si(π)
si_integrand = lambda x: np.sin(x)/x if x != 0 else 1.0
si_pi, _ = integrate.quad(si_integrand, 0, np.pi)
print(f"\n2. Si(π) = {si_pi:.6f}")

# 3. Elliptic integral K(0.5)
k = 0.5
elliptic = lambda theta: 1/np.sqrt(1 - k*np.sin(theta)**2)
K_half, _ = integrate.quad(elliptic, 0, np.pi/2)
print(f"\n3. K(0.5) = {K_half:.6f}")

# 4. Arc length of sin(x)
sin_arc = lambda x: np.sqrt(1 + np.cos(x)**2)
length, _ = integrate.quad(sin_arc, 0, np.pi)
print(f"\n4. Arc length of sin(x) from 0 to π = {length:.6f}")
print(f"   (Straight line would be π = {np.pi:.6f})")

# 5. Expected value of Exp(3)
lam = 3
exp_mean = lambda x: x * lam * np.exp(-lam*x)
mean, _ = integrate.quad(exp_mean, 0, np.inf)
print(f"\n5. E[X] for Exp(3) = {mean:.6f}")
print(f"   Expected (1/λ) = {1/lam:.6f}")

print("\n" + "="*60)
print("All problems solved! ✓")

PRACTICE PROBLEM SOLUTIONS


1. P(Z < 2) = 0.977250
   (About 97.7% of data within 2σ)

2. Si(π) = 1.851937

3. K(0.5) = 1.854075

4. Arc length of sin(x) from 0 to π = 3.820198
   (Straight line would be π = 3.141593)

5. E[X] for Exp(3) = 0.333333
   Expected (1/λ) = 0.333333

All problems solved! ✓
